# 深層ニューラルネットワーク・正則化手法・高度な最適化アルゴリズム

このノートブックでは、深層ニューラルネットワーク、正則化手法、高度な最適化アルゴリズムのハンズオン学習を提供します。深層ニューラルネットワークの構築、訓練、評価を行いながら、パフォーマンスを向上させるための様々なテクニックを実装していきます。

## 深層学習の基礎概念

### 機械学習と深層学習

**機械学習**とは、コンピュータがデータから学習し、明示的なプログラミングなしにタスクを改善する能力を指します。従来のプログラミングでは、私たちがルールとデータを提供し、コンピュータが答えを出力します。一方、機械学習では、データと答えを提供し、コンピュータ自身がルールを学習します。

**深層学習**は機械学習の一種で、多層のニューラルネットワーク（深層ニューラルネットワーク）を使用して、データから表現を学習します。人間の脳の構造に着想を得たこの手法は、以下のような特徴を持ちます：

- **階層的な特徴学習**: 低レベルの特徴（エッジ、色など）から徐々に高レベルの特徴（顔、物体など）を自動的に学習します
- **表現学習**: 生データから有用な表現を自動的に発見します
- **エンド・ツー・エンド学習**: 前処理から最終出力まで一貫した学習が可能です

### なぜ深層学習なのか？

深層学習は以下のような利点があります：

1. **高い表現力**: 複雑なパターンや非線形関係を学習する能力があります
2. **特徴エンジニアリングの自動化**: 手動で特徴を設計する労力を削減します
3. **スケーラビリティ**: より多くのデータとより大きなモデルで性能が向上し続けます
4. **転移学習**: あるタスクで学習した知識を別のタスクに転用できます

### このチュートリアルの目的

このチュートリアルでは、単にモデルを構築するだけでなく、深層学習の基本原理を理解し、モデル性能を向上させるための様々な技術を学びます。特に以下の点に焦点を当てています：

- 深層ニューラルネットワークの基本構造と動作原理の理解
- 過学習を防ぐための正則化手法の理解と実装
- モデル学習の効率と効果を高める最適化アルゴリズムの比較
- モデル評価と解釈のベストプラクティス

これらの概念と技術を習得することで、実践的な深層学習モデルを開発するための強固な基盤を築くことができるでしょう。

## 必要なライブラリのインポート

深層学習を実装するために必要な主要ライブラリを以下でインポートします：

- **NumPy**: 数値計算のための基本ライブラリ
- **matplotlib**: データの可視化ツール
- **TensorFlow/Keras**: 深層学習モデルの構築と訓練のためのフレームワーク
- **pandas**: データ操作と分析用のライブラリ
- **seaborn**: 統計的なデータ可視化のためのライブラリ

また、日本語でのグラフ表示を可能にするために `japanize-matplotlib` を使用します。

In [ ]:
# 必要なライブラリをインポート
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import pandas as pd
import seaborn as sns

# TensorFlowのバージョンを確認
print(f"TensorFlow バージョン: {tf.__version__}")

%matplotlib inline

%pip install -U japanize-matplotlib

# 日本語化ライブラリのインポート＆有効化
import japanize_matplotlib
japanize_matplotlib.japanize()

import matplotlib.font_manager as fm

# フォントファミリを Meiryo に
plt.rcParams['font.family']       = 'Meiryo'
plt.rcParams['axes.unicode_minus'] = False

plt.style.use('ggplot')
sns.set(font_scale=1.2)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import japanize_matplotlib
import sys
import matplotlib.font_manager as fm

# Check if running on Linux
if sys.platform.startswith('linux'):
    # List available fonts with Japanese support
    fonts = [f for f in fm.findSystemFonts() if 'gothic' in f.lower() or 'mincho' in f.lower() or 'meiryo' in f.lower()]
    
    if fonts:
        # Use the first available Japanese font
        plt.rcParams['font.family'] = fm.FontProperties(fname=fonts[0]).get_name()
    else:
        # Fallback to IPAGothic or another common Japanese font
        plt.rcParams['font.family'] = 'IPAGothic, Noto Sans CJK JP, MS Gothic'
else:
    # On Windows/Mac, use platform-specific fonts
    if sys.platform.startswith('win'):
        plt.rcParams['font.family'] = 'MS Gothic'
    elif sys.platform.startswith('darwin'):
        plt.rcParams['font.family'] = 'AppleGothic'

# Disable font warnings (optional)
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

plt.rcParams['axes.unicode_minus'] = False
japanize_matplotlib.japanize()  # Apply japanize_matplotlib settings

## データセットの読み込みと準備

このチュートリアルでは、深層学習実験の入門としてMNISTデータセットを使用します。

### MNISTデータセットとは？

**MNIST**は「Modified National Institute of Standards and Technology」の略で、手書き数字（0〜9）の画像データセットです。機械学習、特に画像認識の入門として広く使われています。

**特徴：**
- 60,000枚のトレーニング画像と10,000枚のテスト画像を含みます
- 各画像は28×28ピクセルのグレースケール画像（0〜255の輝度値）
- 各画像には0〜9の数字ラベルが付いています

### データの前処理ステップ

機械学習モデルにデータを投入する前に、いくつかの重要な前処理ステップを実行する必要があります：

1. **正規化（Normalization）**: ピクセル値を0〜1の範囲に変換します。これはモデルのパフォーマンスと収束速度を向上させるために重要です。
   - 元のピクセル値範囲：0〜255
   - 正規化後の範囲：0〜1

2. **データの平坦化（Flattening）**: 2次元の画像データ（28×28）を1次元ベクトル（784）に変換します。全結合層（Dense層）は1次元入力を想定しているため、この変換が必要となります。
   - 元の形状：(サンプル数, 28, 28)
   - 平坦化後：(サンプル数, 784)

3. **ワンホットエンコーディング**: カテゴリラベルを二値クラス行列に変換します。
   - 元のラベル：0〜9の単一の数値（例：「5」）
   - 変換後：10次元のバイナリベクトル（例：[0,0,0,0,0,1,0,0,0,0]）

以下のコードでは、これらの前処理ステップを具体的に実装しています。

In [ ]:
# MNISTデータセットを読み込む
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# ピクセル値を0から1の間に正規化
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# 全結合層で使用するためにデータを平坦化
x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)

# クラスベクトルをバイナリクラス行列に変換（ワンホットエンコーディング）
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(f"訓練サンプル数: {x_train.shape[0]}")
print(f"テストサンプル数: {x_test.shape[0]}")
print(f"入力形状: {x_train.shape[1]}")

### データの可視化

前処理したデータを理解するために、いくつかのサンプル画像を可視化してみましょう。これにより、モデルが実際に何を「見ている」のかをより具体的に理解できます。

In [ ]:
# オリジナルデータを再度読み込み（可視化のためにのみ使用）
(orig_x_train, orig_y_train), _ = keras.datasets.mnist.load_data()

# ランダムに9つの画像を選択して表示
plt.figure(figsize=(10, 10))
for i in range(9):
    # ランダムインデックスを生成
    idx = np.random.randint(0, orig_x_train.shape[0])
    
    plt.subplot(3, 3, i+1)
    plt.imshow(orig_x_train[idx], cmap='gray')
    plt.title(f"ラベル: {orig_y_train[idx]}")
    plt.axis('off')
    
plt.tight_layout()
plt.suptitle("MNISTデータセットのサンプル画像", fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()

# 前処理（平坦化と正規化）後のデータの形状を確認
print(f"訓練データの形状: {x_train.shape}")
print(f"ワンホットエンコード後のラベル例: {y_train[0]}")

## 深層ニューラルネットワークの構築

Keras Sequential APIを使用して、基本的な深層ニューラルネットワークのアーキテクチャを定義しましょう。複数の隠れ層と適切な活性化関数を持つモデルを作成します。

### ニューラルネットワークの基本構造

ニューラルネットワークは、以下の主要な要素で構成されています：

1. **入力層 (Input Layer)**: データがモデルに入る最初の層です。各ノードは入力特徴量の1つの値に対応します。
   - 今回のケース：784ノード（28×28ピクセルの画像を平坦化したもの）

2. **隠れ層 (Hidden Layers)**: 入力層と出力層の間にある層で、複雑なパターンを学習します。
   - 層の数が多いほど、より複雑な関係を学習できますが、過学習のリスクも高まります。
   - 今回は512→256→128→64ノードと徐々に小さくしていくアーキテクチャを採用しています。

3. **出力層 (Output Layer)**: モデルの最終層で、タスクに応じた形式で結果を提供します。
   - 多クラス分類（今回のケース）：クラス数に対応するノード（10個）を持ち、softmax活性化関数を使用

### 活性化関数 (Activation Functions)

活性化関数は、ニューロンの出力に非線形性を導入する数学的関数です：

1. **ReLU (Rectified Linear Unit)**: `f(x) = max(0, x)`
   - 隠れ層でよく使用される
   - 利点：計算が高速で、勾配消失問題を軽減
   - 今回の隠れ層では全てこの活性化関数を採用

2. **Softmax**: 複数クラスの確率分布を出力する関数
   - 多クラス分類の出力層で使用される
   - 全ての出力値の合計は1になる（確率として解釈可能）
   - 今回の出力層で使用

### Kerasでのモデル構築方法

Kerasでは主に2つのAPIを使ってモデルを構築できます：

1. **Sequential API**: 層を順番に積み重ねるシンプルなモデルに適しています。
   - 使いやすく、初心者に推奨
   - 線形的なネットワークアーキテクチャに最適

2. **Functional API**: より複雑なモデルアーキテクチャを構築できます。
   - マルチ入力/マルチ出力モデル
   - 層を共有するモデル
   - 非線形トポロジー（DAG）のモデル

以下のコードでは、まずSequential APIを使って基本的なDNNを構築し、その後同じアーキテクチャをFunctional APIで実装します。

In [ ]:
def create_basic_dnn():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(784,)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

# モデルを作成
basic_model = create_basic_dnn()

# モデルをコンパイル
basic_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# モデルの概要を表示
basic_model.summary()

### モデル概要の読み方

上記の`model.summary()`の出力について解説します：

- **Layer (type)**: レイヤーの名前とタイプ
- **Output Shape**: 各レイヤーの出力テンソルの形状
  - (None, 512) の「None」はバッチサイズを表し、可変であることを示します
- **Param #**: レイヤーの学習可能なパラメータ数
  - 例えば最初の層では、784（入力）× 512（出力）+ 512（バイアス）= 401,920 パラメータ

### モデルのコンパイル

モデルをコンパイルする際に指定する主要な要素：

1. **損失関数 (Loss Function)**: モデルの予測と実際のターゲット値との差を測定
   - 分類タスク用: `categorical_crossentropy`（多クラス）または`binary_crossentropy`（二値）
   - 回帰タスク用: `mean_squared_error`など

2. **オプティマイザ (Optimizer)**: パラメータ更新の方法を決定
   - `adam`: 最も一般的で効果的なオプティマイザの1つ
   - 他に`sgd`, `rmsprop`などがあります（後で詳しく学びます）

3. **評価指標 (Metrics)**: モデルの性能を監視する指標
   - `accuracy`: 分類タスクでの正確さを測定
   - 他に`precision`, `recall`, `f1`などがあります

同じモデルアーキテクチャをより柔軟性の高いファンクショナルAPIを使って実装してみましょう：

In [ ]:
def create_functional_dnn():
    inputs = Input(shape=(784,))
    x = Dense(512, activation='relu')(inputs)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# ファンクショナルAPIを使用してモデルを作成
functional_model = create_functional_dnn()

# モデルをコンパイル
functional_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# モデルの概要を表示
functional_model.summary()

### Sequential APIとFunctional APIの比較

上記の2つの実装を比較すると：

1. **Sequential API**:
   - より直感的でシンプル
   - コードが短く、読みやすい
   - 単純な線形モデルに最適

2. **Functional API**:
   - より柔軟で、複雑なアーキテクチャを構築可能
   - 各層を変数として扱うため、後で参照したり再利用したりできる
   - マルチ入力/出力モデルやスキップ接続などの高度な機能をサポート

今回は単純な順次接続モデルなので、どちらのAPIを使っても結果は同じです。より複雑なモデル（ResNet, U-Net, SiameseNetなど）を実装する場合は、Functional APIが必要になることが多いでしょう。

## 正則化テクニックの適用

過学習を防ぐためのさまざまな正則化テクニックを探ってみましょう：

1. L1正則化（Lasso）- 重みの絶対値にペナルティを課すことでスパース性を促進します
2. L2正則化（Ridge）- 重みの二乗和を加えることで大きな重みにペナルティを課します
3. ドロップアウト - トレーニング中にランダムに一部の入力ユニットを0に設定します

これらの正則化テクニックを使用したモデルを作成し、それらのパフォーマンスを比較しましょう。

### 過学習（オーバーフィッティング）の問題

深層学習モデルを訓練する際に直面する主要な課題の1つが**過学習**です。これは、モデルが訓練データに対して過度に最適化され、新しいデータに対する汎化性能が低下する現象です。

**過学習の兆候**：
- 訓練データでの性能は非常に良いが、検証/テストデータでの性能が悪い
- 訓練誤差と検証誤差の間に大きな差がある
- 複雑なモデル（多くのパラメータを持つモデル）ほど過学習しやすい

### 正則化とは？

**正則化**とは、モデルの複雑さに制約を加えることで過学習を防ぎ、汎化性能を向上させる技術です。モデルを「よりシンプルに」保つことを促進します。

### L1正則化（Lasso正則化）

**仕組み**：重みの絶対値の和にペナルティを課します。

**数学的表現**：損失関数 + λ * Σ|w|（λは正則化の強度を制御するハイパーパラメータ）

**特徴**：
- **スパース性の促進**: 多くの重みを正確に0にする傾向があります
- **特徴選択**: 重要でない特徴を自動的に除外します
- **解釈可能性**: スパースなモデルはより解釈しやすい

**適用例**：特徴量が多く、一部が冗長または不要な場合に有効

### L2正則化（Ridge正則化）

**仕組み**：重みの二乗和にペナルティを課します。

**数学的表現**：損失関数 + λ * Σw²

**特徴**：
- **重みの縮小**: すべての重みを小さくしますが、完全に0にはしません
- **安定性**: 相関のある特徴に対して安定した解を提供します
- **より滑らかな解**: 極端な重みを避けます

**適用例**：マルチコ線形性（特徴間の高い相関）がある場合に効果的

### ドロップアウト

**仕組み**：トレーニング中にランダムにニューロンを「ドロップアウト」（一時的に無効化）します。

**具体的な動作**：
- 各訓練バッチで、指定された割合（例：30%）のニューロンがランダムに選ばれ、その出力が0に設定されます
- 推論時には、全てのニューロンが活性化され、訓練時との出力スケールの差を調整するために重みが調整されます

**特徴**：
- **アンサンブル効果**: 複数のサブネットワークを同時に訓練するような効果があります
- **共適応の防止**: ニューロンが特定のニューロンに過度に依存するのを防ぎます
- **ノイズへの頑健性**: ランダムノイズを加えることで、より堅牢なモデルになります

**適用例**：大規模なニューラルネットワークで特に効果的

### 複数の正則化手法の組み合わせ

実際には、複数の正則化テクニックを組み合わせることで、より効果的に過学習を防ぐことができます。例えば、L2正則化とドロップアウトを組み合わせると：

- L2正則化は重みの大きさを制限
- ドロップアウトはニューロン間の依存関係を減少

これらは互いに補完し合い、より堅牢なモデルを作成するのに役立ちます。以下のコードでは、これらの正則化テクニックを実装したモデルを作成します。

In [ ]:
# L1正則化を使用したモデル
def create_l1_model():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(784,), kernel_regularizer=l1(0.001)),
        Dense(256, activation='relu', kernel_regularizer=l1(0.001)),
        Dense(128, activation='relu', kernel_regularizer=l1(0.001)),
        Dense(64, activation='relu', kernel_regularizer=l1(0.001)),
        Dense(10, activation='softmax')
    ])
    return model

# L2正則化を使用したモデル
def create_l2_model():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(784,), kernel_regularizer=l2(0.001)),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(10, activation='softmax')
    ])
    return model

# ドロップアウトを使用したモデル
def create_dropout_model():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(784,)),
        Dropout(0.3),  # 30%のドロップアウト率
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(10, activation='softmax')
    ])
    return model

# L2正則化とドロップアウトの両方を使用したモデル
def create_combined_reg_model():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(784,), kernel_regularizer=l2(0.001)),
        Dropout(0.3),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.3),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(10, activation='softmax')
    ])
    return model

# モデルを作成してコンパイルする
l1_model = create_l1_model()
l1_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

l2_model = create_l2_model()
l2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

dropout_model = create_dropout_model()
dropout_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

combined_model = create_combined_reg_model()
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# モデルの概要を表示
combined_model.summary()

### 正則化パラメータの選択

正則化の効果は、正則化パラメータ（λ値やドロップアウト率）に大きく依存します：

- **λが小さすぎると**: 正則化の効果が弱く、過学習を防げない可能性があります
- **λが大きすぎると**: モデルが単純化されすぎて、訓練データにも適合できなくなる「過少適合（アンダーフィッティング）」が発生する可能性があります
- **ドロップアウト率が低すぎると**: 効果が薄くなります
- **ドロップアウト率が高すぎると**: 情報の流れが妨げられ、モデルの学習能力が低下します

最適な正則化パラメータは、交差検証（Cross-validation）などを使って実験的に決定するのが一般的です。問題やデータセットによって最適値は異なります。

## 高度な最適化アルゴリズムの実装

さまざまな最適化アルゴリズムとテクニックを探っていきましょう：
1. 標準的な確率的勾配降下法（Standard SGD）
2. モーメンタム付きSGD（SGD with momentum）
3. RMSprop
4. Adam
5. 学習率スケジューリング（Learning rate scheduling）

これらの異なる最適化手法を用いてモデルを訓練し、収束性能とパフォーマンスを比較してみましょう。

### 最適化アルゴリズムの基本概念

**最適化アルゴリズム**は、モデルの損失関数を最小化するために、モデルのパラメータ（重みとバイアス）をどのように更新するかを決定するアルゴリズムです。深層学習における最適化は、複雑で非凸な高次元空間での最小値探索という難しい問題です。

### 勾配降下法の基本

すべての最適化アルゴリズムは、**勾配降下法**の基本概念に基づいています。勾配降下法では、現在のパラメータ位置での損失関数の勾配（傾き）を計算し、その勾配の方向に逆らってパラメータを更新します。

基本的な更新式：  
θ = θ - η∇J(θ)  
ここで、  
- θ：モデルパラメータ  
- η：学習率（ステップサイズ）  
- ∇J(θ)：θに関する損失関数の勾配  

### 確率的勾配降下法（SGD）

**確率的勾配降下法（SGD）** は、各パラメータ更新にトレーニングデータの小さなランダムサブセット（ミニバッチ）を使用します。

**特徴**：
- **単純**: 実装が簡単で計算コストが低い
- **ノイズ**: バッチが異なるため更新が揺らぎ、局所的最小値から抜け出す可能性がある
- **課題**: 収束が遅く、最適なパスを見つけるのに苦労することがある（「ジグザグ」パターンで進む）

### モーメンタム付きSGD

**モーメンタム**は、以前の更新方向の「慣性」を加えることで通常のSGDを拡張します。物理学における慣性の概念に着想を得ています。

更新式：  
v = γv - η∇J(θ)  
θ = θ + v  
ここで、v はベロシティ（速度）ベクトル、γ はモーメンタム係数（通常0.9前後）

**利点**：
- **収束の加速**: 勾配が同じ方向を指す場合、更新幅が大きくなる
- **振動の軽減**: 勾配が方向を変える場合、ジグザグパターンが減少
- **局所最小値からの脱出**: 慣性により小さな局所的最小値を超えられる可能性

### RMSprop

**RMSprop** (Root Mean Square Propagation) は、パラメータごとに異なる学習率を適応的に調整するアルゴリズムです。

更新式：  
E[g²]t = 0.9E[g²]t-1 + 0.1g²t  
θ = θ - η / √(E[g²]t + ε) * gt  
ここで、E[g²]t は過去の勾配二乗の移動平均、ε は小さな値（ゼロ除算防止）

**利点**：
- **適応的学習率**: 勾配が大きいパラメータは小さく更新し、小さいパラメータは大きく更新する
- **対称性の解消**: 特徴のスケールが異なる場合に有効
- **振動の抑制**: 急な次元での更新を抑制する

### Adam（Adaptive Moment Estimation）

**Adam**は、モーメンタムとRMSpropの利点を組み合わせた最適化手法です。2つのモーメント（一次モーメント：平均、二次モーメント：分散）を計算します。

簡略化した更新式：  
m = β1m + (1-β1)g  # 勾配の移動平均（一次モーメント）  
v = β2v + (1-β2)g²  # 勾配二乗の移動平均（二次モーメント）  
θ = θ - η * m / (√v + ε)  # バイアス補正後の更新  

**利点**：
- **収束性**: 通常、他の最適化手法より速く収束する
- **メモリ効率**: 追加のメモリ消費が少ない
- **ハイパーパラメータ頑健性**: デフォルトのハイパーパラメータが多くの場合でうまく機能する
- **現在の事実上の標準**: 多くの深層学習タスクでデフォルトの選択肢となっている

### 学習率スケジューリング

**学習率スケジューリング**は、トレーニング中に学習率を動的に調整する技術です。

**主な戦略**：

1. **ステップ減衰**: 特定のエポック数後に学習率を段階的に減少させる（このノートブックで使用）
2. **指数減衰**: 各エポックで学習率を指数関数的に減少させる（例：lr = lr0 * e^(-kt)）
3. **1/t減衰**: 学習率をエポック数に反比例して減少させる（例：lr = lr0 / (1 + kt)）
4. **コサイン減衰**: 学習率をコサイン関数に従って減少させる
5. **ウォームアップ**: 小さな学習率から始め、徐々に増加させた後、再び減少させる

**利点**：
- **初期の高速収束**: 最初は大きな学習率で素早く収束
- **後期の微調整**: 後半は小さな学習率で細かく調整し、最適解に近づける
- **振動の軽減**: トレーニング後期の振動を抑制する

### 最適化アルゴリズムの選択ガイド

- **初めての場合**: Adamを選択（汎用性が高く、多くの場合でうまく機能する）
- **計算リソースが限られている**: SGDまたはモーメンタム付きSGDを検討
- **過学習が懸念される**: 学習率スケジューリングを追加
- **最良の性能を求める**: 複数の最適化手法を試し、検証セットで比較

以下のコードでは、これらの異なる最適化アルゴリズムを実装しています。

In [ ]:
# 最適化手法の比較用にシンプルなモデルを作成する関数
def create_simple_model():
    model = Sequential([
        Dense(256, activation='relu', input_shape=(784,)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

# 学習率スケジューラを定義
def lr_schedule(epoch):
    initial_lr = 0.001
    if epoch < 5:
        return initial_lr
    elif epoch < 10:
        return initial_lr * 0.5
    else:
        return initial_lr * 0.1

lr_callback = LearningRateScheduler(lr_schedule)

# 異なるオプティマイザでモデルを作成
# 標準SGD
sgd_model = create_simple_model()
sgd_model.compile(
    optimizer=SGD(learning_rate=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# モーメンタム付きSGD
sgd_momentum_model = create_simple_model()
sgd_momentum_model.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# RMSprop
rmsprop_model = create_simple_model()
rmsprop_model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Adam
adam_model = create_simple_model()
adam_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 学習率スケジューリング付きAdam
adam_lr_model = create_simple_model()
adam_lr_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

### 最適化アルゴリズムの実験と可視化

以下では、学習中の振る舞いを理解するために、一部のモデルをトレーニングし、それらのパフォーマンスを比較します。完全な比較のためには、すべてのモデルをトレーニングしてみることもできますが、時間を節約するために、ここではAdamと学習率スケジューリング付きAdamのみを比較します。

**データ準備のポイント**:
- トレーニング高速化のために、元の訓練データセットの一部（10,000サンプル）のみを使用します
- トレーニング中のモデル性能をリアルタイムで監視するために、別の検証セット（2,000サンプル）を用意します
- バッチサイズ128でエポック数10でトレーニングします

In [ ]:
# 迅速なデモンストレーションのために小さなサブセットを使用
subset_size = 10000
x_train_subset = x_train[:subset_size]
y_train_subset = y_train[:subset_size]
x_val = x_train[subset_size:subset_size+2000]
y_val = y_train[subset_size:subset_size+2000]

# Adamモデルをトレーニング
adam_history = adam_model.fit(
    x_train_subset, y_train_subset,
    epochs=10,
    batch_size=128,
    validation_data=(x_val, y_val),
    verbose=1
)

# 学習率スケジューリング付きAdamモデルをトレーニング
adam_lr_history = adam_lr_model.fit(
    x_train_subset, y_train_subset,
    epochs=10,
    batch_size=128,
    validation_data=(x_val, y_val),
    callbacks=[lr_callback],
    verbose=1
)

## モデルのパフォーマンス評価

モデルを評価し、学習曲線を通して訓練プロセスを可視化してみましょう。

### モデル評価の重要性

モデル評価は機械学習プロセスの重要な部分です。評価することで以下のような疑問に答えられます：

- モデルはどれくらい正確に予測できるか？
- モデルは過学習（オーバーフィッティング）しているか？
- どのモデル設計や最適化手法が最も効果的か？
- モデルはさらなる改善が必要か？

### 学習曲線の解釈

**学習曲線（Learning Curves）** は、訓練の進行に伴うモデル性能の変化を示すグラフです。以下のポイントに注目することで多くの洞察が得られます：

1. **訓練精度と検証精度の比較**:
   - 訓練精度 > 検証精度（大きな差）→ 過学習の可能性大
   - 訓練精度 ≈ 検証精度 → モデルは適切に汎化している
   - 両方の精度が低い → 過少適合（アンダーフィッティング）

2. **学習曲線の形状**:
   - 平坦化しているか？ → モデルが学習の上限に達している
   - 依然として上昇/下降しているか？ → さらに訓練することで改善の可能性あり
   - 不安定か？ → 学習率が高すぎるか、バッチサイズが小さすぎる可能性

3. **検証損失の動向**:
   - 増加し始める → 過学習の開始
   - 常に減少 → より良い汎化性能

### 学習率スケジューリングの効果

学習率スケジューリングがモデルのパフォーマンスに与える影響を観察することで、次のような点が理解できます：

- 学習率が低くなると、損失関数の変化が穏やかになる
- 訓練後半での性能向上や安定化
- 学習率の急激な変化は学習曲線の急激な変化として現れることがある

以下のコードでは、異なる最適化戦略（標準Adamと学習率スケジューリング付きAdam）の学習曲線を比較可視化します。

In [ ]:
# 学習曲線をプロットする関数
def plot_learning_curves(histories, labels, metric='accuracy'):
    plt.figure(figsize=(12, 5))
    
    # 訓練メトリクスのプロット
    plt.subplot(1, 2, 1)
    for i, history in enumerate(histories):
        plt.plot(history.history[metric], label=f'{labels[i]} 訓練')
    plt.title(f'訓練 {metric.capitalize()}')
    plt.xlabel('エポック')
    plt.ylabel(metric.capitalize())
    plt.legend()
    
    # 検証メトリクスのプロット
    plt.subplot(1, 2, 2)
    for i, history in enumerate(histories):
        plt.plot(history.history[f'val_{metric}'], label=f'{labels[i]} 検証')
    plt.title(f'検証 {metric.capitalize()}')
    plt.xlabel('エポック')
    plt.ylabel(metric.capitalize())
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 訓練と検証の損失をプロット
    plt.figure(figsize=(12, 5))
    
    # 訓練損失のプロット
    plt.subplot(1, 2, 1)
    for i, history in enumerate(histories):
        plt.plot(history.history['loss'], label=f'{labels[i]} 訓練')
    plt.title('訓練損失')
    plt.xlabel('エポック')
    plt.ylabel('損失')
    plt.legend()
    
    # 検証損失のプロット
    plt.subplot(1, 2, 2)
    for i, history in enumerate(histories):
        plt.plot(history.history['val_loss'], label=f'{labels[i]} 検証')
    plt.title('検証損失')
    plt.xlabel('エポック')
    plt.ylabel('損失')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# AdamとLRスケジューリング付きAdamの学習曲線をプロット
plot_learning_curves(
    [adam_history, adam_lr_history],
    ['Adam', 'LRスケジューリング付きAdam']
)

正則化モデルの1つをトレーニングし、テストセットでその性能を評価してみましょう：

### テストセット評価の意義

モデル開発の最終段階では、訓練や検証に使用していない独立したデータセット（**テストセット**）でモデルを評価することが重要です。これにより、モデルの実際の汎化性能（未知のデータに対する性能）を評価できます。

テストセットは以下の特徴を持つべきです：

- 訓練・検証データと同じ分布から抽出されたもの
- モデル開発中には一切使用していないもの（モデルにとって完全に「未知」であること）
- 将来のデータを適切に代表するもの

### 予測結果の視覚化

モデルの予測を視覚化することで、以下のような洞察が得られます：

- どのような入力で誤分類が発生しやすいか
- 誤分類のパターンや傾向があるか（例：特定の数字が別の数字と混同されやすいなど）
- モデルの信頼度（確率出力）はどの程度か

以下のコードでは、L2正則化とドロップアウトを組み合わせたモデルを訓練し、テストセットでの性能を評価します。さらに、いくつかのテストサンプルとその予測結果を可視化します。

In [ ]:
# 組み合わせ正則化モデルをトレーニング
combined_history = combined_model.fit(
    x_train_subset, y_train_subset,
    epochs=15,
    batch_size=128,
    validation_data=(x_val, y_val),
    verbose=1
)

# テストセットで評価
test_scores = combined_model.evaluate(x_test, y_test, verbose=1)
print(f"テスト精度: {test_scores[1]:.4f}")

# テストセットで予測を行う
y_pred = combined_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# いくつかのサンプル画像と予測結果を表示
plt.figure(figsize=(15, 6))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title(f"正解: {y_true_classes[i]}, 予測: {y_pred_classes[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

### 混同行列（Confusion Matrix）で詳細な分析

混同行列は、分類モデルの性能をより詳細に分析するのに役立つツールです。各クラスについて予測されたラベルと実際のラベルの関係を示します。

以下のコードでは、混同行列を作成し可視化します。これにより、どの数字が最も誤分類されやすいか、そしてどのような混同パターンがあるかを識別できます。

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# 混同行列を計算
cm = confusion_matrix(y_true_classes, y_pred_classes)

# 混同行列を可視化
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
plt.xlabel('予測ラベル')
plt.ylabel('真のラベル')
plt.title('混同行列')
plt.show()

# 詳細な分類レポートを表示
print("\n分類レポート:")
print(classification_report(y_true_classes, y_pred_classes, target_names=[str(i) for i in range(10)]))

## まとめと重要なポイント

このノートブックでは、以下の内容について探求しました：

1. Sequential APIとFunctional APIの両方を使用した深層ニューラルネットワークの構築
2. さまざまな正則化テクニックの実装：
   - L1正則化（Lasso）
   - L2正則化（Ridge）
   - ドロップアウト
   - 複数の正則化手法の組み合わせ
   
3. 異なる最適化アルゴリズムの実験：
   - SGD（確率的勾配降下法）
   - モーメンタム付きSGD
   - RMSprop
   - Adam
   - 学習率スケジューリング
   
4. モデルのパフォーマンス評価と学習プロセスの可視化

重要なポイント：
- より深いネットワークはより複雑なパターンを学習できますが、過学習しやすくなります
- 正則化テクニックは過学習を防ぎ、汎化性能を向上させるのに役立ちます
- Adamなどの高度な最適化アルゴリズムは、基本的なSGDよりも通常速く収束します
- 学習率スケジューリングは最適化プロセスの微調整に役立ちます
- 複数の正則化テクニックを組み合わせると、単一のアプローチを使用するよりも良い結果が得られることが多いです

### 次のステップ

深層ニューラルネットワークの基本を理解したところで、以下のような発展的なトピックに取り組むことができます：

1. **畳み込みニューラルネットワーク（CNN）**: 画像処理タスクのための特殊なアーキテクチャ
2. **リカレントニューラルネットワーク（RNN）**: シーケンシャルデータ処理のためのアーキテクチャ
3. **転移学習**: 事前訓練されたモデルを使用して、少ないデータでも高性能なモデルを構築
4. **ハイパーパラメータチューニング**: グリッドサーチやランダムサーチを使用したモデルの最適化
5. **モデルの解釈可能性**: モデルの予測がなぜそのような結果になるかを理解するための技術

ニューラルネットワークの学習と実装を続けながら、実際の問題に適用することで理解を深めていきましょう。